In [1]:
import umap
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"
res_folders=os.listdir('../../results/')
#model_folder='/home/mara/multitask_adversarial/results/NCOUNT_822/'
CONCEPT=['domain']
import keras
keras.__version__
from sklearn.metrics import accuracy_score
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.visible_device_list = str(0)# str(hvd.local_rank())
keras.backend.set_session(tf.Session(config=config))
verbose=1 
init=tf.global_variables_initializer() #initialize_all_variables()
sess=tf.Session()
sess.run(init)
#reducer = umap.UMAP()
#reducer = reducer.fit(feats)
#embedding = reducer.transform(feats)

Using TensorFlow backend.


In [2]:
CONCEPT

['domain']

In [3]:
model_folder='/home/mara/multitask_adversarial/results/DOMAIN_822/'
BASEL_FOLDER='../../results/BASEL_1139/best_model.h5'

In [4]:
## Loading OS libraries to configure server preferences
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings("ignore")
import setproctitle
SERVER_NAME = 'ultrafast'
EXPERIMENT_TYPE='test_domain'
import time
import sys
import shutil
## Adding PROCESS_UC1 utilities
sys.path.append('../../lib/TASK_2_UC1/')
from models import *
from util import otsu_thresholding
from extract_xml import *
from functions import *                   
sys.path.append('../../lib/')
from mlta import *
import math
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.visible_device_list = str(0)# str(hvd.local_rank())
keras.backend.set_session(tf.Session(config=config))

verbose=1 
"""loading dataset files"""
#rank = MPI.COMM_WORLD.rank
cam16 = hd.File('/home/mara/adversarialMICCAI/data/ultrafast/cam16_500/patches.h5py',  'r', libver='latest', swmr=True)
all500 = hd.File('/home/mara/adversarialMICCAI/data/ultrafast/all500/patches.h5py',  'r', libver='latest', swmr=True)
extra17 = hd.File('/home/mara/adversarialMICCAI/data/ultrafast/extra17/patches.h5py','r', libver='latest', swmr=True)
tumor_extra17=hd.File('/home/mara/adversarialMICCAI/data/ultrafast/1129-1155/patches.h5py', 'r', libver='latest', swmr=True)
test2 = hd.File('/mnt/nas2/results/IntermediateResults/Camelyon/ultrafast/test_data2/patches.hdf5', 'r', libver='latest', swmr=True)
pannuke= hd.File('/mnt/nas2/results/IntermediateResults/Camelyon/pannuke/patches_fix.hdf5', 'r', libver='latest', swmr=True)

global datasetss
datasetss={'cam16':cam16,'all500':all500,'extra17':extra17, 'tumor_extra17':tumor_extra17, 'test_data2': test2, 'pannuke':pannuke}

global concept_db
concept_db = hd.File('../../data/normalized_cmeasures/concept_values_def.h5py','r')

#SYSTEM CONFIGS 
CONFIG_FILE = '../../doc/config.cfg'
COLOR = True
BATCH_SIZE = 32

# SAVE FOLD
f=open(model_folder+"/seed.txt","r")
seed=1001#int(f.read())
if verbose:  print(seed)
#f.write(str(seed))
f.close()


# SET PROCESS TITLE
setproctitle.setproctitle('{}'.format(EXPERIMENT_TYPE))

# SET SEED
np.random.seed(seed)
tf.set_random_seed(seed)

# DATA SPLIT CSVs 
train_csv=open('/mnt/nas2/results/IntermediateResults/Camelyon/train_shuffle.csv', 'r') # How is the encoding of .csv files ?
val_csv=open('/mnt/nas2/results/IntermediateResults/Camelyon/val_shuffle.csv', 'r')
test_csv=open('/mnt/nas2/results/IntermediateResults/Camelyon/test_shuffle.csv', 'r')
train_list=train_csv.readlines()
val_list=val_csv.readlines()
test_list=test_csv.readlines()
test2_csv = open('/mnt/nas2/results/IntermediateResults/Camelyon/test2_shuffle.csv', 'r')
test2_list=test2_csv.readlines()
test2_csv.close()
train_csv.close()
val_csv.close()
test_csv.close()
data_csv=open('../../doc/pannuke_data_shuffle.csv', 'r')
data_list=data_csv.readlines()
data_csv.close()

# STAIN NORMALIZATION
def get_normalizer(patch, save_folder='../../results/'):
    normalizer = ReinhardNormalizer()
    normalizer.fit(patch)
    np.save('{}/normalizer'.format(save_folder),normalizer)
    np.save('{}/normalizing_patch'.format(save_folder), patch)
    #print('Normalisers saved to disk.')
    return normalizer
def normalize_patch(patch, normalizer):
    return np.float64(normalizer.transform(np.uint8(patch)))

global normalizer
db_name, entry_path, patch_no = get_keys(data_list[0])
normalization_reference_patch = datasetss[db_name][entry_path][patch_no]
normalizer = get_normalizer(normalization_reference_patch, save_folder='../../results/')

"""
Batch generators: 
They load a patch list: a list of file names and paths. 
They use the list to create a batch of 32 samples. 
"""

# Retrieve Concept Measures
def get_concept_measure(db_name, entry_path, patch_no, measure_type=''):
    if measure_type=='domain':
        return get_domain(db_name, entry_path)
    path=db_name+'/'+entry_path+'/'+str(patch_no)+'/'+measure_type.strip(' ')
    try:
        cm=concept_db[path][0]
        return cm
    except:
        print("[ERR]: {}, {}, {}, {} with path {}".format(db_name, entry_path, patch_no, measure_type, path))
        #import pdb; pdb.set_trace()
        return 1.
    
# BATCH GENERATORS
import keras.utils
class DataGenerator(keras.utils.Sequence):
    def __init__(self, patch_list, concept=CONCEPT, batch_size=32, shuffle=True, data_type=0):
        self.batch_size=batch_size
        self.patch_list=patch_list
        self.shuffle=shuffle
        self.concept = concept
        self.data_type=data_type
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.floor(len(self.patch_list)/self.batch_size))
    
    def __getitem__(self, index):
        indexes=self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        patch_list_temp=[self.patch_list[k] for k in indexes]
        self.patch_list_temp=patch_list_temp
        return self.__data_generation(self), None
    
    def get(self, index):
        indexes=self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        patch_list_temp=[self.patch_list[k] for k in indexes]
        self.patch_list_temp=patch_list_temp
        return self.__data_generation(self), None
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.patch_list))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def __data_generation(self, patch_list_temp):
        patch_list_temp=self.patch_list_temp
        batch_x=np.zeros((len(patch_list_temp), 224,224,3))
        batch_y=np.zeros(len(patch_list_temp))
        i=0
        for line in patch_list_temp:
            db_name, entry_path, patch_no = get_keys(line)
            patch=datasetss[db_name][entry_path][patch_no]
            patch=normalize_patch(patch, normalizer)
            patch=keras.applications.inception_v3.preprocess_input(patch) 
            label = get_class(line, entry_path) 
            if self.data_type!=0:
                label=get_test_label(entry_path)
            batch_x[i]=patch
            batch_y[i]=label
            i+=1
        generator_output=[batch_x, batch_y]
        for c in self.concept:
            batch_concept_values=np.zeros(len(patch_list_temp))
            i=0
            for line in patch_list_temp:
                db_name, entry_path, patch_no = get_keys(line)
                batch_concept_values[i]=get_concept_measure(db_name, entry_path, patch_no, measure_type=c)
                i+=1
            if c=='domain':
                    batch_concept_values=keras.utils.to_categorical(batch_concept_values, num_classes=7)
            generator_output.append(batch_concept_values)
        return generator_output
    

1001
Using brightness standardization


In [5]:
#import matplotlib as mpl   
#mpl.use('Agg')
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings("ignore")
import logging
logging.getLogger('tensorflow').disabled = True
from keras import *
import setproctitle
SERVER_NAME = 'ultrafast'
import time
import sys
import shutil
## Adding PROCESS_UC1 utilities
sys.path.append('../../lib/TASK_2_UC1/')
from models import *
from util import otsu_thresholding
from extract_xml import *
from functions import *                   
sys.path.append('../../lib/')
from mlta import *
import math
import keras.callbacks as callbacks
from keras.callbacks import Callback

In [6]:
keras.backend.clear_session()
"""         
Building guidable model 
"""
def get_baseline_model(hp_lambda=0., domain=False, c_list=[]):
    base_model = keras.applications.inception_v3.InceptionV3(include_top=False, weights='imagenet', input_shape=(224,224,3))
    layers_list=['conv2d_92', 'conv2d_93', 'conv2d_88', 'conv2d_89', 'conv2d_86']
    #layers_list=[]
    for i in range(len(base_model.layers[:])):
        layer=base_model.layers[i]
        if layer.name in layers_list:
            print layer.name
            layer.trainable=True
        else:
            layer.trainable = False
    feature_output=base_model.layers[-1].output
    gap_layer_output = keras.layers.GlobalAveragePooling2D()(feature_output)
    feature_output = Dense(2048, activation='relu', name='finetuned_features1',kernel_regularizer=keras.regularizers.l2(0.01))(gap_layer_output) 
    feature_output = keras.layers.Dropout(0.8, noise_shape=None, seed=None)(feature_output)
    feature_output = Dense(512, activation='relu', name='finetuned_features2',kernel_regularizer=keras.regularizers.l2(0.01))(feature_output)
    feature_output = keras.layers.Dropout(0.8, noise_shape=None, seed=None)(feature_output)
    feature_output = Dense(256, activation='relu', name='finetuned_features3',kernel_regularizer=keras.regularizers.l2(0.01))(feature_output)
    feature_output = keras.layers.Dropout(0.8, noise_shape=None, seed=None)(feature_output)
    grl_layer=GradientReversal(hp_lambda=hp_lambda)
    feature_output_grl = grl_layer(feature_output)
    if domain:
        domain_adversarial = keras.layers.Dense(7, activation = keras.layers.Activation('softmax'), name='domain_adversarial')(feature_output_grl)
    finetuning = Dense(1,name='predictions')(feature_output)
    ## here you need to check how many other concepts you have apart from domain adversarial
    # then you add one layer per each. 
    if domain:
        output_nodes=[finetuning, domain_adversarial]
    else: 
        output_nodes=[finetuning]
    for c in c_list:
        if c!='domain':
            concept_layer=  keras.layers.Dense(1, activation = keras.layers.Activation('linear'), name='extra_{}'.format(c.strip(' ')))(feature_output)
            output_nodes.append(concept_layer)
    model = Model(input=base_model.input, output=output_nodes)
    model.grl_layer=grl_layer
    return model
""" 
Get trainable model with Hepistemic Uncertainty Weighted Loss 
"""
# Custom loss layer
from keras.initializers import Constant
class CustomMultiLossLayer(Layer):
    def __init__(self, nb_outputs=2, **kwargs):
        self.nb_outputs = nb_outputs
        self.is_placeholder = True
        super(CustomMultiLossLayer, self).__init__(**kwargs)
        
    def build(self, input_shape=None):
        self.log_vars = []
        for i in range(self.nb_outputs):
            self.log_vars += [self.add_weight(name='log_var' + str(i), shape=(1,), initializer=Constant(0.), trainable=True)]
        super(CustomMultiLossLayer, self).build(input_shape)
    
    def multi_loss(self,  ys_true, ys_pred):
        #print len(ys_true)
        assert len(ys_true) == self.nb_outputs and len(ys_pred) == self.nb_outputs
        loss = 0
        i=0
        for y_true, y_pred, log_var in zip(ys_true, ys_pred, self.log_vars):
            precision =K.exp(-log_var[0]) ###MODIFICATION HERE
            if i==0:
                pred_loss = bbce(y_true, y_pred)
                term = precision*pred_loss + 0.5 * log_var[0]  
                #term=tf.Print(term, [term], 'bbce: ')
            else:
                pred_loss = keras_mse(y_true, y_pred)
                #pred_loss=tf.Print(pred_loss, [pred_loss], 'MSE: ')
                term = 0.5 * precision * pred_loss + 0.5 * log_var[0]
                #term=tf.Print(term, [term], 'MSE: ')
            loss+=term
            term = 0.
            i+=1
        return K.mean(loss)
    
    def call(self, inputs):
        ys_true = inputs[:self.nb_outputs]
        ys_pred = inputs[self.nb_outputs:]
        loss = self.multi_loss(ys_true, ys_pred)
        self.add_loss(loss, inputs=inputs)
        # We won't actually use the output.
        return K.concatenate(inputs, -1)
def get_trainable_model(baseline_model):
    inp = keras.layers.Input(shape=(224,224,3,), name='inp')
    y1_pred, y2_pred = baseline_model(inp)
    y1_true=keras.layers.Input(shape=(1,),name='y1_true')
    y2_true=keras.layers.Input(shape=(1,),name='y2_true')
    out = CustomMultiLossLayer(nb_outputs=2)([y1_true, y2_true, y1_pred, y2_pred])
    return Model(input=[inp, y1_true, y2_true], output=out)

""" Get trainable model with Hepistemic Uncertainty Weighted Loss """


""" 
LOSS FUNCTIONS
"""
def keras_mse(y_true, y_pred):
    return tf.reduce_mean(tf.keras.losses.mean_squared_error(y_true, y_pred))

def bbce(y_true, y_pred):
    # we use zero weights to set the loss to zero for unlabeled data
    verbose=0
    zero= tf.constant(-1, dtype=tf.float32)
    where = tf.not_equal(y_true, zero)
    where = tf.reshape(where, [-1])
    indices=tf.where(where) #indices where the item of y_true is NOT -1
    indices = tf.reshape(indices, [-1])
    sliced_y_true = tf.nn.embedding_lookup(y_true, indices)
    sliced_y_pred = tf.nn.embedding_lookup(y_pred, indices)
    n1 = tf.shape(indices)[0] #number of train images in batch
    batch_size = tf.shape(y_true)[0]
    n2 = batch_size - n1 #number of test images in batch
    sliced_y_true = tf.reshape(sliced_y_true, [n1, -1])
    n1_ = tf.cast(n1, tf.float32)
    n2_ = tf.cast(n2, tf.float32)
    multiplier = (n1_+ n2_) / n1_
    zero_class = tf.constant(0, dtype=tf.float32)
    where_class_is_zero=tf.cast(tf.reduce_sum(tf.cast(tf.equal(sliced_y_true, zero_class), dtype=tf.float32)), dtype=tf.float32)
    if verbose:
        where_class_is_zero=tf.Print(where_class_is_zero,[where_class_is_zero],'where_class_is_zero: ')
    class_weight_zero = tf.cast(tf.divide(n1_, 2. * tf.cast(where_class_is_zero, dtype=tf.float32)+0.001), dtype=tf.float32)
    
    if verbose:
        class_weight_zero=tf.Print(class_weight_zero,[class_weight_zero],'class_weight_zero: ')
    one_class = tf.constant(1, dtype=tf.float32)
    where_class_is_one=tf.cast(tf.reduce_sum(tf.cast(tf.equal(sliced_y_true, one_class), dtype=tf.float32)), dtype=tf.float32)
    if verbose:
        where_class_is_one=tf.Print(where_class_is_one,[where_class_is_one],'where_class_is_one: ')
        n1_=tf.Print(n1_,[n1_],'n1_: ')
    class_weight_one = tf.cast(tf.divide(n1_, 2. * tf.cast(where_class_is_one,dtype=tf.float32)+0.001), dtype=tf.float32)
    class_weight_zero =  tf.constant(23477.0/(23477.0+123820.0), dtype=tf.float32)
    class_weight_one =  tf.constant(123820.0/(23477.0+123820.0), dtype=tf.float32)
    A = tf.ones(tf.shape(sliced_y_true), dtype=tf.float32) - sliced_y_true 
    A = tf.scalar_mul(class_weight_zero, A)
    B = tf.scalar_mul(class_weight_one, sliced_y_true) 
    class_weight_vector=A+B
    ce = tf.nn.sigmoid_cross_entropy_with_logits(labels=sliced_y_true,logits=sliced_y_pred)
    ce = tf.multiply(class_weight_vector,ce)
    return tf.reduce_mean(ce)

from keras.initializers import Constant
global domain_weight
global main_task_weight

class CustomMultiLossLayer(Layer):
    def __init__(self, new_folder='', nb_outputs=2, **kwargs):
        self.nb_outputs = nb_outputs
        self.is_placeholder = True
        super(CustomMultiLossLayer, self).__init__(**kwargs)
        
    def build(self, input_shape=None):
        # initialise log_vars
        self.log_vars = []
        for i in range(self.nb_outputs):
            self.log_vars += [self.add_weight(name='log_var' + str(i), shape=(1,),
                                              initializer=Constant(0.), trainable=True)]
        super(CustomMultiLossLayer, self).build(input_shape)
    """
    def multi_loss(self, ys_true, ys_pred):
        assert len(ys_true) == self.nb_outputs and len(ys_pred) == self.nb_outputs
        loss = 0
        for y_true, y_pred, log_var in zip(ys_true, ys_pred, self.log_vars):
            precision = K.exp(-log_var[0])
            loss += K.sum(precision * (y_true - y_pred)**2. + log_var[0], -1)
        return K.mean(loss)
    """
    def multi_loss(self,  ys_true, ys_pred):
        assert len(ys_true) == self.nb_outputs and len(ys_pred) == self.nb_outputs
        loss = 0
        i=0
        for y_true, y_pred, log_var in zip(ys_true, ys_pred, self.log_vars):
            precision =keras.backend.exp(-log_var[0]) 
            if i==0:
                pred_loss = bbce(y_true, y_pred)
                term = main_task_weight*precision*pred_loss + main_task_weight*0.5 * log_var[0]  
                #term=tf.Print(keras.backend.mean(term), [keras.backend.mean(term)], 'mean bbce: ')
            #elif i==1:
                # I need to find a better way for this
            #    pred_loss = keras.losses.categorical_crossentropy(y_true, y_pred)
                #keras_mse(y_true, y_pred)
            #    term =  domain_weight * precision * pred_loss + domain_weight * log_var[0]
                #term=tf.Print(keras.backend.mean(term), [keras.backend.mean(term)], 'mean cce: ')
            else:
                pred_loss = keras_mse(y_true, y_pred)
                #pred_loss=tf.Print(pred_loss, [pred_loss], 'MSE: ')
                term = 0.5 * precision * pred_loss + 0.5 * log_var[0]
            loss+=term
            term = 0.
            i+=1
        return keras.backend.mean(loss)
    
    def call(self, inputs):
        ys_true = inputs[:self.nb_outputs]
        ys_pred = inputs[self.nb_outputs:]
        loss = self.multi_loss(ys_true, ys_pred)
        self.add_loss(loss, inputs=inputs)
        return keras.backend.concatenate(inputs, -1)

"""
EVALUATION FUNCTIONs
"""
def accuracy_domain(y_true,y_pred):
    y_p_r=np.round(y_pred)
    acc = np.equal(y_p_r, y_true)**1.
    acc = np.mean(np.float32(acc))
    return acc
def my_sigmoid(x):
    return 1 / (1 + np.exp(-x))
def my_accuracy_np(y_true, y_pred):
    sliced_y_pred = my_sigmoid(y_pred)
    y_pred_rounded = np.round(sliced_y_pred)
    acc = np.equal(y_pred_rounded, y_true)**1.
    acc = np.mean(np.float32(acc))
    return acc
def r_square_np(y_true, y_pred):
    SS_res =  np.sum(np.square(y_true - y_pred))
    SS_tot = np.sum(np.square(y_true - np.mean(y_true)))
    r2_mine=( 1 - SS_res/(SS_tot + keras.backend.epsilon()) )
    return ( 1 - SS_res/(SS_tot + keras.backend.epsilon()) )

global report_val_acc 
global report_val_r2
global report_val_mse
report_val_acc=[]
report_val_r2=[]
report_val_mse=[]

In [7]:
hp_lambda=0.
c_list=CONCEPT
base_model = keras.applications.inception_v3.InceptionV3(include_top=False, weights='imagenet', input_shape=(224,224,3))
layers_list=['conv2d_92', 'conv2d_93', 'conv2d_88', 'conv2d_89', 'conv2d_86']
#layers_list=[]
for i in range(len(base_model.layers[:])):
    layer=base_model.layers[i]
    if layer.name in layers_list:
        print layer.name
        layer.trainable=True
    else:
        layer.trainable = False
feature_output=base_model.layers[-1].output
gap_layer_output = keras.layers.GlobalAveragePooling2D()(feature_output)
feature_output = keras.layers.Dense(2048, activation='relu', name='finetuned_features1',kernel_regularizer=keras.regularizers.l2(0.01))(gap_layer_output) 
feature_output = keras.layers.Dropout(0.8, noise_shape=None, seed=None)(feature_output)
feature_output = keras.layers.Dense(512, activation='relu', name='finetuned_features2',kernel_regularizer=keras.regularizers.l2(0.01))(feature_output)
feature_output = keras.layers.Dropout(0.8, noise_shape=None, seed=None)(feature_output)
feature_output = keras.layers.Dense(256, activation='relu', name='finetuned_features3',kernel_regularizer=keras.regularizers.l2(0.01))(feature_output)
#feature_output = keras.layers.Dropout(0.8, noise_shape=None, seed=None)(feature_output)
feature_output = keras.layers.Dropout(0.8, noise_shape=None, seed=None)(feature_output)
grl_layer=GradientReversal(hp_lambda=hp_lambda)
feature_output_grl = grl_layer(feature_output)
domain_adversarial = keras.layers.Dense(7, activation = keras.layers.Activation('softmax'), name='domain_adversarial')(feature_output_grl)
finetuning = Dense(1,name='predictions')(feature_output)
#finetuning = keras.layers.Dense(1,name='predictions')(feature_output)
#sfinetuning = Dense(1,name='predictions')(feature_output)
## here you need to check how many other concepts you have apart from domain adversarial
# then you add one layer per each. 
output_nodes=[finetuning, domain_adversarial]
for c in c_list:
    if c!='domain':
        concept_layer=  keras.layers.Dense(1, activation = keras.layers.Activation('linear'), name='extra_{}'.format(c.strip(' ')))(feature_output)
        output_nodes.append(concept_layer)
model = Model(input=base_model.input, output=output_nodes)
model.grl_layer=grl_layer
#return model
#regression_output = keras.layers.Dense(1, activation = keras.layers.Activation('linear'), name='concept_regressor')(gap_layer_output)
#model = keras.Model(input=base_model.input, output=[finetuning, regression_output])

conv2d_88
conv2d_89
conv2d_92
conv2d_93
conv2d_86


In [8]:
main_task_weight=1. 
domain_weight = 1. #e-100
t_m = get_trainable_model(model)

In [9]:
t_m.load_weights('{}/best_model.h5'.format(model_folder))

ValueError: axes don't match array

In [12]:
test_list

['all500, normal/level7/centre3/patient068/node1/patches, 227\n',
 'tumor_extra17, tumor/level7/centre1/patient034/node3/patches, 185\n',
 'all500, tumor/level7/centre1/patient034/node3/patches, 66\n',
 'all500, normal/level7/centre3/patient068/node1/patches, 401\n',
 'all500, tumor/level7/centre0/patient010/node4/patches, 146\n',
 'tumor_extra17, tumor/level7/centre1/patient034/node3/patches, 327\n',
 'all500, tumor/level7/centre0/patient010/node4/patches, 448\n',
 'all500, normal/level7/centre1/patient034/node3/patches, 45\n',
 'tumor_extra17, tumor/level7/centre1/patient034/node3/patches, 209\n',
 'all500, normal/level7/centre2/patient041/node0/patches, 119\n',
 'tumor_extra17, tumor/level7/centre1/patient034/node3/patches, 167\n',
 'all500, tumor/level7/centre1/patient034/node3/patches, 200\n',
 'tumor_extra17, tumor/level7/centre1/patient034/node3/patches, 338\n',
 'all500, normal/level7/centre2/patient041/node0/patches, 260\n',
 'all500, tumor/level7/centre0/patient010/node4/patc

In [ ]:
baseline=get_baseline_model()
baseline.load_weights(BASEL_FOLDER)

In [ ]:
len(data_list)

In [ ]:
CONCEPT

In [ ]:
type_='def_'
test_list=test_list+test2_list
for layer in ['finetuned_features3']:
    print layer
    get_guided_layer_output = K.function([model.layers[0].input],
                                  [model.get_layer(layer).output])
    get_base_layer_output = K.function([baseline.layers[0].input],
                                  [baseline.get_layer(layer).output])
    N_SAMPLES=len(train_list)
    t_gen=DataGenerator(test_list, concept=CONCEPT, batch_size=BATCH_SIZE, data_type=1)
    
    shape=int(model.get_layer(layer).output.shape[1])
    #import pdb; pdb.set_trace()
    if layer=='mixed10':
        shape=51200
    feats = np.zeros((N_SAMPLES, shape))
    i=0
    labs=np.zeros(N_SAMPLES)
    cvals=np.zeros((N_SAMPLES, 7))
    #idxs = np.random.choice(np.arange(len(data_list)), N_SAMPLES, repetition=False)
    batch_size=32
    while i<=N_SAMPLES:
        #import pdb; pdb.set_trace()
        print i
        input_,_=t_gen.__getitem__(i//batch_size)
        try:
            feats[i:i+batch_size, :] = get_guided_layer_output([input_[0]])[0].reshape(batch_size,-1)
            labs[i:i+batch_size]=input_[1]
            cvals[i:i+batch_size,:]=input_[2]
            i+=batch_size
            
        except:
            print "ERR"
            #import pdb; pdb.set_trace()
            break
    np.save('{}/{}features{}.npy'.format(model_folder,type_, layer), feats)
    np.save('{}/{}labels{}.npy'.format(model_folder, type_,layer), labs)
    np.save('{}/{}cvalues{}.npy'.format(model_folder,type_, layer), cvals)
    
    t_gen=DataGenerator(test_list, concept=CONCEPT, batch_size=BATCH_SIZE, data_type=1)
    #N_SAMPLES=5000
    shape=int(model.get_layer(layer).output.shape[1])
    if layer=='mixed10':
        shape=51200
    feats = np.zeros((N_SAMPLES, shape))
    i=0
    labs=np.zeros(N_SAMPLES)
    cvals=np.zeros((N_SAMPLES,  7))
    batch_size=32
    while i+batch_size<=N_SAMPLES:
        #import pdb; pdb.set_trace()
        print i
        input_,_=t_gen.__getitem__(i//batch_size)
        try:
            feats[i:i+batch_size, :] = get_base_layer_output([input_[0]])[0].reshape(batch_size,-1)
            labs[i:i+batch_size]=input_[1]
            cvals[i:i+batch_size,:]=input_[2]
            i+=batch_size
        except:
            i+=batch_size
            print "ERR"
            break
    np.save('{}/{}base_features{}.npy'.format(model_folder,type_,layer), feats)
    np.save('{}/{}base_labels{}.npy'.format(model_folder,type_,layer), labs)
    np.save('{}/{}base_cvalues{}.npy'.format(model_folder,type_,layer), cvals)


In [ ]:
'{}/{}base_features{}.npy'.format(model_folder,type_,layer)

In [ ]:
cvals

In [ ]:
type_

In [ ]:
layer